In [ ]:
import pandas as pd
import tqdm.auto as tqdm

def read_csv_file(user_id, video_id):
    filepath = f'Experiment_2/{user_id}/video_{video_id}.csv'
    df = pd.read_csv(filepath)
    df['video_id'] = video_id  # 添加video_id字段
    return df

data_with_labels = []

for user_id in range(1, 49):
    user_data = []
    for video_id in range(0, 9):
        df = read_csv_file(user_id, video_id)

        # 对数据进行预处理，例如计算每秒的坐标等
        # df = preprocess(df)

        # 添加标签（用户ID）
        df['user_id'] = user_id

        user_data.append(df)

    user_data_combined = pd.concat(user_data, ignore_index=True)
    data_with_labels.append(user_data_combined)

data_with_labels = pd.concat(data_with_labels, ignore_index=True)


In [ ]:
from datetime import timedelta
import numpy as np
from tqdm.auto import tqdm

def process_data(data):
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    data['rounded_time'] = data['Timestamp'].dt.floor('100ms')

    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    for user_id, user_data in tqdm(data.groupby('user_id')):
        for video_id, video_data in user_data.groupby('video_id'):
            for second, second_data in video_data.groupby(video_data['rounded_time'].dt.floor('1s')):
                time_slices = []
                for _, chunk_data in second_data.groupby(pd.cut(second_data['Timestamp'], pd.date_range(second, second + timedelta(seconds=1), periods=11))):
                    if not chunk_data.empty:
                        chunk_data = chunk_data[['UnitQuaternion.x', 'UnitQuaternion.y', 'UnitQuaternion.z', 'UnitQuaternion.w', 'HmdPosition.x', 'HmdPosition.y', 'HmdPosition.z']]
                        time_slices.append(chunk_data.mean().values)

                if len(time_slices) == 10:
                    if video_id == 'video_0':
                        test_data.append(time_slices)
                        test_labels.append(user_id)
                    else:
                        train_data.append(time_slices)
                        train_labels.append(user_id)

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

train_data, train_labels, test_data, test_labels = process_data(data_with_labels)



In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
# 将用户ID转换为从0开始的整数
unique_labels = np.unique(np.concatenate([train_labels, test_labels]))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
int_train_labels = np.array([label_map[label] for label in train_labels])
int_test_labels = np.array([label_map[label] for label in test_labels])

# 将标签数据转换为one-hot编码
one_hot_train_labels = to_categorical(int_train_labels, num_classes=len(unique_labels))
one_hot_test_labels = to_categorical(int_test_labels, num_classes=len(unique_labels))
def evaluate_accuracy(test_data, test_labels, model):
    with tf.device('/device:GPU:0'):
        predictions = model.predict(test_data)
    predicted_labels = np.argmax(predictions, axis=1)

    correct_predictions = np.sum(predicted_labels == test_labels)
    total_predictions = len(test_labels)
    accuracy = correct_predictions / total_predictions

    return accuracy

model_path = 'result.h5'
loaded_model = load_model(model_path)
accuracy = evaluate_accuracy(test_data, int_test_labels, model)
print(f"预测准确率: {accuracy:.4f}")
